In [ ]:
import panel as pn
import param

In [16]:
"""Check consistency of curated COF database"""
import pandas
import datetime
import os
CURATED_COFS=os.path.abspath('/Users/leopold/Personal/Postdoc-MARVEL/Projects/2019-01-CoRE-COF-paper/updates/CURATED-COFs')
df_papers = pandas.read_csv(os.path.join(CURATED_COFS, 'cof-papers.csv'))

def parse_cofs_id(id):
    return {
        'year': id[1:3],
        'counter': int(id[3:5]),
    }

def mint_paper_id():   

    last_id = sorted(df_papers['CURATED-COFs paper ID'])[-1]
    year = str(datetime.datetime.now().year)
    
    last_dict = parse_cofs_id(last_id)
    
    #import pdb; pdb.set_trace()
    if last_dict['year'] == year[2:]:
        counter = last_dict['counter'] + 1
    else:
        counter = 0
        
    return "p{:s}{:02d}".format(year[2:], counter)


In [17]:
"""From for paper info"""
"CURATED-COFs paper ID","Reference","URL","Title"
"p0500","Science, 2005, 310, 1166-1170","http://science.sciencemag.org/content/310/5751/1166.short","Porous, crystalline, covalent organic frameworks"

pn.extension()

inp_doi = pn.widgets.TextInput(name='Publication DOI', placeholder="10.1021/jacs.9b01891")
btn_doi = pn.widgets.Button(name='Fetch metadata', button_type='primary')

inp_reference = pn.widgets.TextInput(name='Publication reference', placeholder='')
inp_title = pn.widgets.TextInput(name='Publication title', placeholder='')
inp_year = pn.widgets.TextInput(name='Publication year', placeholder='')
inp_paper_id = pn.widgets.TextInput(name='CURATED-COFs paper ID', placeholder='')
inp_url = pn.widgets.TextInput(name='Publication URL', placeholder='')

div_out = pn.widgets.StaticText(name='Output', value='')






def get_metadata(doi):
    import requests
    import json
    response = requests.get(url="http://dx.doi.org/"+doi, 
            headers={"accept":"application/rdf+xml;q=0.5, application/vnd.citationstyles.csl+json;q=10"})
    return json.loads(response.content)

def on_click_fetch(event):
    import dateutil
    from crossref.restful import Works
    
    works = Works()
    metadata = works.doi(inp_doi.value or "10.1021/jacs.9b01891")
    inp_title.value = str(metadata['title'][0])
    
    year = str(dateutil.parser.parse(metadata['indexed']['date-time']).year)
    inp_year.value = year
    
    journal = str(metadata['container-title'][0])
    inp_reference.value = "{}, {}, {}, {}".format(journal, year, metadata['volume'], metadata['page'])
    
    inp_url.value = "https://doi.org/" + inp_doi.value
    
    inp_paper_id.value = mint_paper_id()
        
btn_doi.on_click(on_click_fetch)

column = pn.Column(pn.Row(inp_doi, btn_doi))
column.append(inp_title)
column.append(inp_year)
column.append(inp_reference)
column.append(inp_url)
column.append(inp_paper_id)
column.servable()

Column
    [0] Row
        [0] TextInput(name='Publication DOI', placeholder='10.1126/science.169.3946....)
        [1] Button(button_type='primary', name='Fetch metadata')
    [1] TextInput(name='Publication title')
    [2] TextInput(name='Publication year')
    [3] TextInput(name='Publication reference')
    [4] TextInput(name='Publication URL')
    [5] TextInput(name='CURATED-COFs paper ID')

In [3]:
# class DetailView(param.Parameterized):
#     doi = param.TextInput(name='Publication DOI', placeholder='10.')
    
# #     x = param.Selector(objects=plot_dict, default='PE', doc="abcd")
# #     y = param.Selector(objects=plot_dict, default='Pur')
# #     clr = param.Selector(objects=OrderedDict(plot_dict, **{'Bond type': 'bond_type'}), default='Td')
#     msg = pn.pane.HTML("")
#     label=get_structure_from_url()
#     _plot = None  # reference to current plot
    
#     @param.depends()
#     def iso_plot(self):
#         from detail.plot_isotherm import plot_isotherm
#         label = self.label
#         pm_co2 = get_data_aiida(structure_label=label, link_label='iso_co2')
#         pm_n2 = get_data_aiida(structure_label=label, link_label='iso_n2')
#         mpl_fig = plot_isotherm(pm_co2=pm_co2, pm_n2=pm_n2, label=label)

#         column = pn.Column(get_title("Isotherms for " + self.label, uuid=pm_co2.uuid))
#         column.append(pn.pane.Matplotlib(mpl_fig, dpi=150, width=700))

#         return column

#     @param.depends()
#     def ener_plot(self):
#         from detail.plot_cp2k import plot_energy_steps
#         steps_node=get_data_aiida(structure_label=self.label, link_label='cp2k_stepsfile')
#         steps_file = steps_node.get_file_abs_path()

#         column = pn.Column(get_title("Robust cell optimization of " + self.label, uuid=steps_node.uuid))
#         column.append(pn.pane.Bokeh(plot_energy_steps(steps_file, self.label)))
#         return column

#     @param.depends()
#     def structure_ref(self):
#         from detail.structure import structure_jsmol
#         cif_node = get_data_aiida(structure_label=self.label, link_label='ref_cif_qeq')
#         fpath = cif_node.get_file_abs_path()
#         with open(fpath, 'r') as handle:
#             cif_str = handle.read()
            
#         column = pn.Column(get_title("Reported structure for " + self.label, uuid=cif_node.uuid))
#         column.append(pn.pane.Bokeh(structure_jsmol(cif_str=cif_str)))
#         return column
  
#     @param.depends()
#     def structure_opt(self):
#         from detail.structure import structure_jsmol
#         cif_node = get_data_aiida(structure_label=self.label, link_label='opt_cif_ddec')
#         fpath = cif_node.get_file_abs_path()
#         with open(fpath, 'r') as handle:
#             cif_str = handle.read()
            
#         column = pn.Column(get_title("DFT-optimized structure for " + self.label, uuid=cif_node.uuid))
#         column.append(pn.pane.Bokeh(structure_jsmol(cif_str=cif_str)))
#         return column
    
#     @param.depends()
#     def process(self):
#         from detail.process import print_process
#         process_node =  get_data_aiida(structure_label=self.label, link_label='opt_out_pe')
#         process_dict = process_node.get_dict()
#         column = pn.Column(get_title("Process sketch for " + self.label, uuid=process_node.uuid))
#         column.append(pn.pane.PNG('detail/process.png', width=800))
#         column.append(pn.pane.Markdown(print_process(process_node)))
#         return column
 
#     @param.depends()
#     def graph(self):
#         from detail.graph import get_graph
#         column = pn.Column(PlainSVG(get_graph(self.label)))
#         #column = pn.Column(pn.pane.SVG(get_graph(self.label)))
#         return column

#     @param.depends(self.doi.value)
#     def metadata(self, doi):
        
#         column = pn.Column(self.doi)
#         column.append(pn.widgets.Button(name='Fetch', button_type='primary'))
#         return column

        
# explorer = DetailView()

# # gspec = pn.GridSpec(sizing_mode='stretch_both', max_width=900)
# # gspec[0, 0] = explorer.structure
# # gspec[1, 0] = explorer.ener_plot
# # gspec[2, 0] = explorer.iso_plot
# # gspec.servable()

# tabs = pn.Tabs()
# tabs.extend([
#     ("Metadata", explorer.metadata),
# #   ('Reported structure', explorer.structure_ref),
# #     ('DFT-optimization energy', explorer.ener_plot),
# #     ('DFT-optimized structure', explorer.structure_opt),
# #     ('Isotherms', explorer.iso_plot),
# #     ('CCS process', explorer.process),
# ])

# pn.extension()
# tabs.servable()

In [ ]:
"""From for CIF info"""
"CURATED-COFs ID","Source","Name","Dimension","Elements","Reference","Duplicate found","Modifications"
"05000N2","tong-v2, 035","COF-1","2D","H,B,C,O","Science, 2005, 310, 1166-1170","none","none"

inp_source = pn.widgets.Select(name='CIF source', options=['tong-v2', 'SI', 'CSD'])
inp_name = pn.widgets.TextInput(name='CIF name', placeholder='As used in publication')
# Tested this with panel 0.7.0a14, but loading the app was very slow
# inp_cif = pn.widgets.input.TextAreaInput(name='CIF', value="""
# data_crystal
#  
# _cell_length_a    
# _cell_length_b    
# _cell_length_c    
# _cell_angle_alpha
# _cell_angle_beta  
# _cell_angle_gamma 
# 
# _symmetry_space_group_name_H-M  'P 1'
#  
# loop_
# _atom_site_label
# _atom_site_fract_x
# _atom_site_fract_y
# _atom_site_fract_z
# """)
inp_cif = pn.widgets.FileInput(name='CIF')

btn_cif = pn.widgets.Button(name='Parse CIF', button_type='primary')
def on_click_parse(event):
    from ase.io import read
    from io import BytesIO
    import re
    
    content = BytesIO(inp_cif.value)
    atoms = read(content, format='cif')
    formula = atoms.get_chemical_formula()
    elements = [e for e in re.split(r'\d+', formula) if e]
    inp_elements.value = ", ".join(elements)
    
    display(inp_cif.value)
        
btn_cif.on_click(on_click_parse)


def display(cif_string):
    from submit.structure import structure_jsmol
    column2.append(pn.pane.Bokeh(structure_jsmol(cif_string)))


inp_elements = pn.widgets.TextInput(name='CIF elements', placeholder='C,H,...')
inp_dimensionality = pn.widgets.Select(name='CIF dimensionality', options=['2D', '3D'])
inp_duplicate = pn.widgets.TextInput(name='CIF duplicate found', value='none')
inp_modifications = pn.widgets.TextInput(name='CIF modifications', placeholder='xy alignment, replicated along z, ...')

column2 = pn.Column(inp_reference)
column2.append(inp_source)
column2.append(inp_name)
column2.append(pn.Row(inp_cif, btn_cif))
column2.append(inp_elements)
column2.append(inp_dimensionality)
column2.append(inp_duplicate)
column2.append(inp_modifications)


column2.servable()

